## Evaluate transfer learning on Turnedtable Watertank (Dataset 2)  using an SVM Classifier: Self-supervised approach 

In [1]:
import tensorflow as tf
import h5py
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import sys
import os
sys.path.append('../../../')
sys.path.append('../../')

### Configure Tensorflow for GPU device

In [2]:
tf.config.experimental_run_functions_eagerly(True)
print("[INFO] Tensorflow Version:", tf.__version__)

if tf.config.list_physical_devices("GPU") and tf.test.is_built_with_cuda():
    print("[INFO] Tensorflow built with CUDA")
    print("[INFO] Number GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    print("[INFO] List of GPU devices:", tf.config.list_physical_devices("GPU"))
    physical_devices = tf.config.list_physical_devices("GPU")
    # tf.config.experimental.set_memory_growth(physical_devices[0], True)
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)

else:
    print("[ERROR] GPU not detected, make sure tensorflow-gpu is installed and that GPU is recognized")
    exit()

[INFO] Tensorflow Version: 2.2.0
[INFO] Tensorflow built with CUDA
[INFO] Number GPUs Available:  1
[INFO] List of GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Define utilities

In [3]:
# utilities
def flatten(x):
    return x.reshape((x.shape[0], -1))

def classSampling(X, y, samplesPerClass, numberOfClasses):
    X_ret = np.zeros((samplesPerClass * numberOfClasses, X.shape[1]), dtype = np.float32)
    y_ret = np.zeros((samplesPerClass * numberOfClasses), dtype = np.uint8)
    count = 0

    for classIdx in range(numberOfClasses):
        indices = np.where(y == classIdx)[0]

        #if len(indices) < samplesPerClass:
        #    raise IndexError("Not enough samples for class {} to produce {} samples per class. Only {} class samples available".format(classIdx, samplesPerClass, len(indices)))

        doResample = len(indices) < samplesPerClass

        chosenIndices = np.random.choice(indices, samplesPerClass, replace = doResample)

        for ci in chosenIndices:
            X_ret[count] = X[ci]
            y_ret[count] = y[ci]

            count += 1

    return X_ret, y_ret

### Define dataset and dataloader

In [4]:
class SonarTurnedTableSupervised(object):
    def __init__(self, file_path):
        self.file_path = file_path

    def _normalize_images(self, images):
        """
        Normalize sonar images by 1/255.
        """
        return [element/255.0 for element in images]

    def get_sonar_data(self):
        """
        Reads from HDF5 file containing sonar data (resized to fix dims).
        Returns list of np arrays containing image data.
        """

        print("[INFO] Retrieving Sonar Turned Table Supervised Data")

        with h5py.File(self.file_path, "r") as f:
            # list all groups
            print("hdf5 dataset keys: %s" % f.keys())

            # get images and labels
            x_train = f["x_train"][...].astype(np.float32)
            y_train = f["y_train"][...]

            x_test = f["x_test"][...].astype(np.float32)
            y_test = f["y_test"][...]

            _, x_val, _, y_val = train_test_split(x_test, y_test, train_size=0.5)

            print("[INFO] Data dimensions")
            print("Train", len(x_train))
            print("Val", len(x_val))
            print("Test", len(x_test))

            # matias normalization
            # multiply by 255 because hdf5 file comes as 1/255
            x_train *= 255.0
            x_val *= 255.0
            x_test *= 255.0

            x_train -= 84.51
            x_val -= 84.51
            x_test  -= 84.51

        return (x_train, y_train), (x_val, y_val), (x_test, y_test)
    
def load_sonar_turnedtable_supervised(file_path):
    """
    Loads test data from turnedtable dataset.
    """
    print()
    print("[INFO] Loading Tenorflow dataset")

    dataset_object = SonarTurnedTableSupervised(file_path)

    # Read data
    (x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset_object.get_sonar_data()

    # Train data
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(len(x_train))
    train_dataset = train_dataset.prefetch(25)

    # Validation data
    # val_dataset = tf.data.Dataset.from_tensor_slices((x_val, labels_val))
    # val_dataset = val_dataset.shuffle(buffer_size=len(x_val)).batch(batch_size)
    # val_dataset = val_dataset.prefetch(25)

    # Test data
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_dataset = test_dataset.shuffle(buffer_size=len(x_test)).batch(len(x_test)) # feed full test set
    test_dataset = test_dataset.prefetch(25)

    print()
    print("[INFO] Tensorflow data dimensions")
    # print(train_dataset)
    # print(val_dataset)
    print(test_dataset)

    # return train_dataset, val_dataset, test_dataset
    return train_dataset, test_dataset

###  Load Pretrained Model

In [3]:
from architectures.resnet20 import resnet20_model

PRETRAINED_NUM_CLASSES = 11 # 11 supervised, 4 self-supervised
input_shape = [96, 96, 1]

model_name = "resnet20"
pretraining_mode = "supervised_learning"
layers = ["flatten", "activation_93", "activation_91"] # TODO: check layer names

model = resnet20_model(input_shape, PRETRAINED_NUM_CLASSES)

model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

In [6]:
# check weights BEFORE loading pretrained model (second conv layer)
print(model.layers[2].get_weights()[0])

[[[[-0.04997722 -0.00918527  0.01357953  0.08176917  0.06000052
    -0.00139114 -0.0724242   0.03291108  0.03121746  0.04279524
    -0.01553656  0.13586219  0.00316131 -0.01334774 -0.02930988
    -0.05185268 -0.05685161  0.13629852  0.11300887 -0.02623674
     0.0049383   0.04281893 -0.11455145  0.00488639  0.09588464
    -0.09362297 -0.00974336  0.12212704 -0.09049764 -0.0637119
    -0.07018055  0.01287864]]

  [[-0.08207813  0.14111169 -0.09982266  0.00216188 -0.00656706
    -0.14206333 -0.02288432  0.03056547  0.09128149  0.0546158
    -0.03057618  0.06981447 -0.02749737 -0.08946207 -0.01168334
     0.02585427  0.05832204  0.10934396 -0.06381647 -0.12536232
     0.0932765  -0.10132461  0.0696253   0.00136797 -0.08722375
     0.09293519 -0.13446206  0.07858631  0.04757109 -0.11601305
     0.13609232  0.11480694]]

  [[ 0.13478278 -0.01185457  0.00201017 -0.04251099  0.03003304
    -0.11473129 -0.00936019 -0.00282069  0.09222922 -0.14126159
    -0.03163882 -0.00796644 -0.00715981  0.0

In [7]:
# path to pretrained model checkpoint
pretrained_checkpoint_prefix = os.path.join("../../../pretraining/results/" + pretraining_mode + "/checkpoints/sonar1/" + model_name + "/batch_size_128/96x96_substract_mean_online_aug_width_32")

# optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.5, epsilon=1)

# define pretrained checkpoint model
checkpoint_pretrained = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, model=model) # 4 ssl, 11 sl
manager_pretrained = tf.train.CheckpointManager(checkpoint_pretrained, pretrained_checkpoint_prefix, max_to_keep=3)

# restore model weights
checkpoint_pretrained.restore(manager_pretrained.latest_checkpoint)

if manager_pretrained.latest_checkpoint:
    print("[INFO] Pretrained checkpoint restored correctly: {}".format(manager_pretrained.latest_checkpoint))
else:
    print("[INFO] Could not restore pretrained checkpoint correctly, make sure path to pre-trained folder is correct.")

[INFO] Pretrained checkpoint restored correctly: ../../../pretraining/results/self_supervised_learning/checkpoints/sonar1/mobilenet/batch_size_128/96x96_substract_mean_online_aug_width_32/ckpt-23


In [8]:
# check weights AFTER loading pretrained model (second conv layer)
print(model.layers[2].get_weights()[0])

[[[[ 0.14404939 -0.09309183  0.05454894 -0.06229801 -0.06013727
    -0.07732987  0.07269716 -0.03821912 -0.11697425  0.1388517
     0.08845436  0.14909086 -0.0434094   0.07132739  0.04855403
    -0.1459768  -0.04156217  0.12690915  0.11585397 -0.02719531
    -0.13449992 -0.04256891  0.1360308  -0.1615945   0.12728855
     0.0988554  -0.02976339 -0.02112121 -0.12318154 -0.07028557
     0.01778408 -0.00416138]]

  [[ 0.00133723 -0.00801746 -0.12868382 -0.13219982  0.10902002
    -0.06519885  0.12391113 -0.0785736   0.11305472 -0.05531718
     0.0411387   0.0567715   0.06359521 -0.0998892  -0.1435193
    -0.10393097 -0.01258101  0.15248203 -0.11573518  0.07517115
    -0.0860673   0.0008493   0.0494826  -0.10693439 -0.01119721
    -0.04008756 -0.09663434 -0.04191499  0.0678202   0.03121695
    -0.15428929 -0.06734885]]

  [[-0.12454671 -0.10658687 -0.02881929 -0.14558476 -0.07411117
    -0.04570181  0.00137575  0.00256155  0.14861685  0.12742507
     0.06354919  0.09335297  0.01787042  0.0

### Define models up to intermediate layers


In [9]:
print("Intermediate layers from model:", layers)

Intermediate layers from model: ['conv_pw_11_relu', 'flatten', 'conv_pw_12_relu']


In [12]:
# https://stackoverflow.com/questions/63297838/how-can-i-obtain-the-output-of-an-intermediate-layer-feature-extraction
model_flatten = tf.keras.Model(inputs=model.get_layer("input_1").output, 
                               outputs=model.get_layer(layers[0]).output)

model_activation_93 = tf.keras.Model(inputs=model.get_layer("input_1").output, 
                                     outputs=model.get_layer(layers[1]).output)

model_activation_91 = tf.keras.Model(inputs=model.get_layer("input_1").output, 
                                     outputs=model.get_layer(layers[2]).output)



### Generate vector embeddings for train and test data (up to n-th layer)

In [13]:
# define tensorflow dataset
data_dir = "../../../../../../datasets/sonar_turntable_dataset_2/marine-debris-turntable-classification-object_classes-platform-96x96.hdf5"
train_dataset, test_dataset = load_sonar_turnedtable_supervised(data_dir)

# load tensorflow tensors individually
x_train, y_train = next(iter(train_dataset))
x_test, y_test = next(iter(test_dataset))


[INFO] Loading Tenorflow dataset
[INFO] Retrieving Sonar Turned Table Supervised Data
hdf5 dataset keys: <KeysViewHDF5 ['class_names', 'x_test', 'x_train', 'y_test', 'y_train']>
[INFO] Data dimensions
Train 1505
Val 323
Test 645

[INFO] Tensorflow data dimensions
<PrefetchDataset shapes: ((None, 96, 96, 1), (None,)), types: (tf.float32, tf.int64)>


In [14]:
# perform a forward pass to generate embeddings (both train and test data) (for each n-th layer)

# NOTE: when doing this all tensors are loaded into memory, this saturates NVIDIA memory (nvidia-smi)
x_train_flatten = model_flatten([x_train], training=False)
x_train_activation_93 = model_activation_93([x_train], training=False)
x_train_activation_91 = model_activation_91([x_train], training=False)

x_test_flatten = model_flatten([x_test], training=False)
x_test_activation_93 = model_activation_93([x_test], training=False)
x_test_activation_91 = model_activation_91([x_test], training=False)


In [15]:
x_train_flatten

<tf.Tensor: shape=(1505, 6, 6, 512), dtype=float32, numpy=
array([[[[1.21386254e+00, 4.89381775e-02, 8.45410645e-01, ...,
          0.00000000e+00, 4.96995270e-01, 9.05933157e-02],
         [1.40980411e+00, 1.15261090e+00, 8.35694194e-01, ...,
          0.00000000e+00, 5.93474329e-01, 4.82767463e-01],
         [1.82574883e-01, 6.56372130e-01, 5.60133994e-01, ...,
          0.00000000e+00, 6.67990267e-01, 9.42384839e-01],
         [1.29431516e-01, 1.25769377e+00, 2.52908558e-01, ...,
          0.00000000e+00, 5.47911167e-01, 1.50075305e+00],
         [0.00000000e+00, 7.05164850e-01, 0.00000000e+00, ...,
          0.00000000e+00, 3.08221430e-01, 7.03265250e-01],
         [1.43316180e-01, 8.97599578e-01, 6.38682246e-01, ...,
          0.00000000e+00, 0.00000000e+00, 6.05568409e-01]],

        [[1.58779538e+00, 2.70380199e-01, 6.62188411e-01, ...,
          0.00000000e+00, 9.18517351e-01, 0.00000000e+00],
         [1.36984611e+00, 1.52387643e+00, 6.65007174e-01, ...,
          5.64474225e-

### Transfer Learning setup: classification with subsamples per object class (few shot learning)

In [16]:
# transfer learning params
# SAMPLES_PER_CLASS = [1, 5, 10, 20, 30, 40, 50] # NOTE: taking more samples per class since it is 88 for 50
# SAMPLES_PER_CLASS = [10, 20, 30, 40, 50, 70, 90, 110, 130, 150, len(x_test)]
SAMPLES_PER_CLASS = [10, 20, 30, 40, 50, 80, 110, 140, 170, 200]
TRIALS = 10

NUM_CLASSES_WATERTANK = 11
NUM_CLASSES_TURNEDTABLE = 12

In [17]:
# Flatten train & test data for SVM
x_train_flatten = flatten(x_train_flatten.numpy())
x_train_activation_93 = flatten(x_train_activation_93.numpy())
x_train_activation_91 = flatten(x_train_activation_91.numpy())

x_test_flatten = flatten(x_test_flatten.numpy())
x_test_activation_93 = flatten(x_test_activation_93.numpy())
x_test_activation_91 = flatten(x_test_activation_91.numpy())

In [18]:
# these two are not modified (only x_test, x_train)
print(y_train.numpy().shape)
print(y_test.numpy().shape)

y_train = y_train.numpy() # convert from tf tensor --> numpy
y_test = y_test.numpy()

(1505,)
(645,)


### Run svm tl evaluation with spc for each n-th layer

In [19]:
def train_svm_with_spc(x_train, y_train, x_test, y_test):
    """
    Takes embeddings from pretrained model and evaluates transfer learning 
    with few samples per class.
    """
    # NOTE: svm takes original labels (not one-hot encoding)
    for spc in SAMPLES_PER_CLASS:
        accuracies = []

        for i in range(TRIALS):
            x_sample, y_sample = classSampling(x_train, y_train, spc, NUM_CLASSES_TURNEDTABLE)

            svm = SVC(C=1.0, decision_function_shape = 'ovo', kernel="linear")
            svm.fit(x_sample, y_sample)

            train_acc = svm.score(x_sample, y_sample)
            test_acc = svm.score(x_test, y_test)

            print("SPC {} Train Accuracy: {:.3f}".format(spc, train_acc))
            print("SPC {} Test Accuracy: {:.3f}".format(spc, test_acc))
            print()

            accuracies.append(test_acc)

        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)

        mean_acc = round(100 * mean_acc, 3)
        std_acc = round(100 * std_acc, 3)

        print("After {} trials - Test Accuracy is {} +- {}".format(TRIALS, mean_acc, std_acc ))
        print("------------------------------------------------------------------------------")
        print()

In [ ]:
train_svm_with_spc(x_train_flatten, y_train, x_test_flatten, y_test)

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.594

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.586

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.580

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.552

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.605

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.600

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.572

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.597

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.577

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.560

After 10 trials - Test Accuracy is 58.217 +- 1.657
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.679

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.653

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.678

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.653

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.651


In [20]:
train_svm_with_spc(x_train_activation_93, y_train, x_test_activation_93, y_test)


SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.620

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.681

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.713

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.691

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.688

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.690

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.662

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.699

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.712

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.729

After 10 trials - Test Accuracy is 68.853 +- 2.884
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.812

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.795

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.795

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.805

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.775


In [21]:
train_svm_with_spc(x_train_activation_91, y_train, x_test_activation_91, y_test)


SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.648

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.684

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.628

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.670

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.647

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.625

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.662

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.679

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.640

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.569

After 10 trials - Test Accuracy is 64.512 +- 3.174
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.722

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.738

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.744

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.707

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.718
